In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_excel('spacestri.xlsx')
print(df.head())

   id             nom  date_ouverture  date_fermeture                 website  \
0  71         La loge            2012             NaN          www.la-loge.be   
1   8  Jeune création            2006             NaN                     NaN   
2  72       The ister            2011             NaN         www.theister.be   
3  73         Komplot            2002             NaN            www.kmplt.be   
4  74         Abilene            2011             NaN  www.abilenegallery.com   

                        email                             adresse horaires  \
0             info@la-loge.be  Kluisstraat - rue de l’Ermitage 86      NaN   
1     jeunecreation@gmail.com                       24 rue Berthe      NaN   
2            info@theister.be                rue Vandenbranden 42      NaN   
3               info@kmplt.be           295 Avenue Van Volxemlaan      NaN   
4  contact@abilenegallery.com              Rue de la Victoire 163      NaN   

  presentation responsables  ...            

In [18]:
# Ajouter les colonnes pour Question 1, Question 2, Réponse Q1, Réponse Q2
df['question_1'] = ''
df['question_2'] = ''
df['reponse_q1'] = ''
df['reponse_q2'] = ''

In [19]:
print(df.head())

   id             nom  date_ouverture  date_fermeture                 website  \
0  71         La loge            2012             NaN          www.la-loge.be   
1   8  Jeune création            2006             NaN                     NaN   
2  72       The ister            2011             NaN         www.theister.be   
3  73         Komplot            2002             NaN            www.kmplt.be   
4  74         Abilene            2011             NaN  www.abilenegallery.com   

                        email                             adresse horaires  \
0             info@la-loge.be  Kluisstraat - rue de l’Ermitage 86      NaN   
1     jeunecreation@gmail.com                       24 rue Berthe      NaN   
2            info@theister.be                rue Vandenbranden 42      NaN   
3               info@kmplt.be           295 Avenue Van Volxemlaan      NaN   
4  contact@abilenegallery.com              Rue de la Victoire 163      NaN   

  presentation responsables  ... longitude  

In [11]:
import pandas as pd
import re

# Charger la base de données à partir de l'Excel
df = pd.read_excel('spacestri.xlsx')

# Charger le fichier texte extrait du PDF
with open('extracted_text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Fonction pour extraire les noms et dates du texte PDF
def extraire_noms_dates(texte):
    # Expression régulière pour extraire les noms suivis de dates entre parenthèses
    pattern = r'([A-Za-z0-9\s\+\-\(\)éÉû\-&’]*)\s*\((\d{4}\s*-\s*\d{0,4}|\d{4}\s*-\s* )\)'
    matches = re.findall(pattern, texte)
    # Retourne seulement les noms sans les dates
    noms = [match[0].strip() for match in matches if match[0].strip()]
    return noms

# Extraire les noms seuls du texte PDF
noms_pdf = extraire_noms_dates(text)

# Afficher les noms extraits pour vérification
print("Noms extraits du texte PDF :")
for nom in noms_pdf:
    print(f"- {nom}")

# Créer un dictionnaire pour stocker la correspondance ID-Nom
id_nom_mapping = {}

# Parcourir chaque ligne du DataFrame pour associer nom et ID
for index, row in df.iterrows():
    nom_excel = row['nom'].strip()
    id_ = row['id']
    
    # Chercher une correspondance exacte ou partielle (insensible à la casse)
    for nom_pdf in noms_pdf:
        if nom_excel.lower() == nom_pdf.lower():
            id_nom_mapping[nom_pdf] = id_
            print(f"Correspondance trouvée : ID {id_} pour le nom '{nom_pdf}'")
            break
    else:
        print(f"Aucune correspondance trouvée pour le nom '{nom_excel}'")

# Vérifier les correspondances trouvées
print("\nMapping ID-Nom final :")
for nom, id_ in id_nom_mapping.items():
    print(f"ID {id_} : {nom}")

# Fonction pour extraire les questions et réponses associées à un nom
def extraire_questions_reponses(nom, texte):
    if not nom:  # Vérifier si le nom est vide
        return None, None
    
    # Échapper les caractères spéciaux dans le nom
    nom_escaped = re.escape(nom)
    
    # Pattern regex pour capturer Q1, réponse à Q1, Q2, et réponse à Q2
    # Assurez-vous que le motif correspond exactement à la structure de votre texte
    pattern = rf"{nom_escaped}.*?Q1\s*.*?\n(.*?)Q2\s*.*?\n(.*?)(?=(?:\n[A-Z]|$))"
    match = re.search(pattern, texte, re.DOTALL)
    
    if match:
        reponse1 = match.group(1).strip()  # Réponse Q1
        reponse2 = match.group(2).strip()  # Réponse Q2
        return reponse1, reponse2
    else:
        return None, None

# Ajouter des colonnes pour les questions et réponses
df['question1'] = "Q1: Pouvez-vous nous apporter un témoignage sur la façon dont votre structure répond, à travers ses modalités de fonctionnement, au contexte actuel et/ou celui de son émergence ?"
df['reponse1'] = ""
df['question2'] = "Q2: Pensez-vous que votre espace ou l’espace auquel vous avez participé puisse être considéré comme une œuvre ? Et si oui dans quel sens ?"
df['reponse2'] = ""

# Parcourir les noms dans le mapping pour extraire les réponses
for nom_pdf, id_ in id_nom_mapping.items():
    reponse1, reponse2 = extraire_questions_reponses(nom_pdf, text)
    
    # Afficher les réponses extraites pour vérification
    print(f"\nExtraction des réponses pour ID {id_} - {nom_pdf}:")
    if reponse1:
        print(f"Réponse Q1: {reponse1[:60]}...")  # Afficher les premiers caractères pour éviter un affichage trop long
    else:
        print("Réponse Q1: Non trouvée")
    
    if reponse2:
        print(f"Réponse Q2: {reponse2[:60]}...")
    else:
        print("Réponse Q2: Non trouvée")
    
    # Ajouter les réponses au DataFrame
    if reponse1 and reponse2:
        df.loc[df['id'] == id_, 'reponse1'] = reponse1
        df.loc[df['id'] == id_, 'reponse2'] = reponse2

# Sauvegarder les données mises à jour dans un nouveau fichier Excel
df.to_excel('fichier_mis_a_jour.xlsx', index=False)

print("\nLes données ont été mises à jour et sauvegardées.")


Noms extraits du texte PDF :
- IMMANENCE
- Particules
- Le journal des expositions
- -L’épicerie (1998 - 2004) Donnant-Donnant (2005 - 2007) L’espace d’en bas (2009 - 2019) Les éditions de l’espace d’en bas (2009 - ) Collection adresse
- PAMELA artist-run space
- PeepSpace
- Kurt-forever (2011 - ) 6b
- DISPLAY
- Palais des paris
- Lieu-Commun
- GLASSBOX
- org (2013 - ) ÉCHOTONE
- Poteaux d’angle
- Red Cactus Art Studio
- GLASSBOX-NORD (1997 - ) GLASSBOX-SUD
- DOS MARES
Aucune correspondance trouvée pour le nom 'La loge'
Aucune correspondance trouvée pour le nom 'Jeune création'
Aucune correspondance trouvée pour le nom 'The ister'
Aucune correspondance trouvée pour le nom 'Komplot'
Aucune correspondance trouvée pour le nom 'Abilene'
Aucune correspondance trouvée pour le nom 'Etablissement d'en face'
Aucune correspondance trouvée pour le nom 'Le Commissariat'
Aucune correspondance trouvée pour le nom 'Artists Club Coffre-Fort'
Aucune correspondance trouvée pour le nom '/ 77'
Aucune corr

In [3]:
import pandas as pd
import re
from difflib import get_close_matches

# Charger la base de données à partir de l'Excel
df = pd.read_excel('spacestri.xlsx')

# Charger le fichier texte extrait du PDF
with open('extracted_text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Fonction pour extraire les noms et dates du texte PDF
def extraire_noms_dates(texte):
    # Expression régulière pour extraire les noms suivis de dates entre parenthèses
    pattern = r'([A-Za-z0-9\s\+\-\(\)éÉû\-&’]*)\s*\((\d{4}\s*-\s*\d{0,4}|\d{4}\s*-\s* )\)'
    matches = re.findall(pattern, texte)
    # Retourne seulement les noms sans les dates
    noms = [match[0].strip() for match in matches if match[0].strip()]
    return noms

# Extraire les noms seuls du texte PDF
noms_pdf = extraire_noms_dates(text)

# Fonction pour normaliser les noms
def normaliser_nom(nom):
    # Normaliser le nom pour qu'il commence par une majuscule et le reste en minuscules
    return ' '.join(word.capitalize() for word in nom.split())

# Normaliser les noms extraits
noms_pdf_normalises = [normaliser_nom(re.sub(r'\s+', ' ', nom.lower()).strip()) for nom in noms_pdf]

# Afficher les noms extraits pour vérification
print("Noms extraits du texte PDF :")
for nom in noms_pdf_normalises:
    print(f"- {nom}")

# Créer un dictionnaire pour stocker la correspondance ID-Nom
id_nom_mapping = {}
noms_avec_ids = []  # Liste pour stocker les noms et leurs IDs associés

# Normaliser les noms dans le DataFrame
df['nom_normalise'] = df['nom'].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

# Parcourir chaque ligne du DataFrame pour associer nom et ID
for index, row in df.iterrows():
    nom_excel = row['nom_normalise']
    id_ = row['id']
    
    # Correspondance floue
    matched_nom = get_close_matches(nom_excel, noms_pdf_normalises, n=1, cutoff=0.5)  # Seuil de correspondance à ajuster
    
    if matched_nom:  # Si une correspondance est trouvée
        id_nom_mapping[matched_nom[0]] = id_
        noms_avec_ids.append((matched_nom[0], id_))  # Ajouter le nom et l'ID à la liste
        print(f"Correspondance trouvée : ID {id_} pour le nom '{matched_nom[0]}'")
    else:
        print(f"Aucune correspondance trouvée pour le nom '{row['nom']}'")

# Vérifier les correspondances trouvées
print("\nMapping ID-Nom final :")
for nom, id_ in id_nom_mapping.items():
    print(f"ID {id_} : {nom}")

# Afficher les noms extraits avec leurs IDs associés
print("\nNoms extraits avec leurs IDs associés :")
for nom, id_ in noms_avec_ids:
    print(f"ID {id_} : {nom}")
"""""
# Fonction pour extraire les questions et réponses associées à un nom
def extraire_questions_reponses(nom, texte):
    if not nom:  # Vérifier si le nom est vide
        return None, None
    
    # Échapper les caractères spéciaux dans le nom
    nom_escaped = re.escape(nom)
    
    # Pattern regex pour capturer Q1, réponse à Q1, Q2, et réponse à Q2
    pattern = rf"{nom_escaped}.*?Q1\s*.*?\n(.*?)Q2\s*.*?\n(.*?)(?=(?:\n[A-Z]|$))"
    match = re.search(pattern, texte, re.DOTALL)
    
    if match:
        reponse1 = match.group(1).strip()  # Réponse Q1
        reponse2 = match.group(2).strip()  # Réponse Q2
        return reponse1, reponse2
    else:
        return None, None

# Ajouter des colonnes pour les questions et réponses
df['question1'] = "Q1: Pouvez-vous nous apporter un témoignage sur la façon dont votre structure répond, à travers ses modalités de fonctionnement, au contexte actuel et/ou celui de son émergence ?"
df['reponse1'] = ""
df['question2'] = "Q2: Pensez-vous que votre espace ou l’espace auquel vous avez participé puisse être considéré comme une œuvre ? Et si oui dans quel sens ?"
df['reponse2'] = ""

# Parcourir les noms dans le mapping pour extraire les réponses
for nom_pdf, id_ in id_nom_mapping.items():
    reponse1, reponse2 = extraire_questions_reponses(nom_pdf, text)
    
    # Afficher les réponses extraites pour vérification
    print(f"\nExtraction des réponses pour ID {id_} - {nom_pdf}:")
    if reponse1:
        print(f"Réponse Q1: {reponse1[:60]}...")  # Afficher les premiers caractères pour éviter un affichage trop long
    else:
        print("Réponse Q1: Non trouvée")
    
    if reponse2:
        print(f"Réponse Q2: {reponse2[:60]}...")
    else:
        print("Réponse Q2: Non trouvée")
    
    # Ajouter les réponses au DataFrame
    if reponse1 and reponse2:
        df.loc[df['id'] == id_, 'reponse1'] = reponse1
        df.loc[df['id'] == id_, 'reponse2'] = reponse2

# Sauvegarder les données mises à jour dans un nouveau fichier Excel
df.to_excel('fichier_mis_a_jour.xlsx', index=False)

print("\nLes données ont été mises à jour et sauvegardées.")
"""

Noms extraits du texte PDF :
- Immanence
- Particules
- Le Journal Des Expositions
- L’épicerie (1998 - 2004) Donnant-donnant (2005 - 2007) L’espace D’en Bas (2009 - 2019) Les Éditions De L’espace D’en Bas (2009 - ) Collection Adresse
- Pamela Artist-run Space
- Peepspace
- Kurt-forever (2011 - ) 6b
- Display
- Palais Des Paris
- Lieu-commun
- Glassbox
- Org (2013 - ) Échotone
- Poteaux D’angle
- Red Cactus Art Studio
- Glassbox-nord (1997 - ) Glassbox-sud
- Dos Mares
Aucune correspondance trouvée pour le nom 'La loge'
Aucune correspondance trouvée pour le nom 'Jeune création'
Aucune correspondance trouvée pour le nom 'The ister'
Aucune correspondance trouvée pour le nom 'Komplot'
Aucune correspondance trouvée pour le nom 'Abilene'
Aucune correspondance trouvée pour le nom 'Etablissement d'en face'
Aucune correspondance trouvée pour le nom 'Le Commissariat'
Aucune correspondance trouvée pour le nom 'Artists Club Coffre-Fort'
Aucune correspondance trouvée pour le nom '/ 77'
Aucune corre

'""\n# Fonction pour extraire les questions et réponses associées à un nom\ndef extraire_questions_reponses(nom, texte):\n    if not nom:  # Vérifier si le nom est vide\n        return None, None\n    \n    # Échapper les caractères spéciaux dans le nom\n    nom_escaped = re.escape(nom)\n    \n    # Pattern regex pour capturer Q1, réponse à Q1, Q2, et réponse à Q2\n    pattern = rf"{nom_escaped}.*?Q1\\s*.*?\n(.*?)Q2\\s*.*?\n(.*?)(?=(?:\n[A-Z]|$))"\n    match = re.search(pattern, texte, re.DOTALL)\n    \n    if match:\n        reponse1 = match.group(1).strip()  # Réponse Q1\n        reponse2 = match.group(2).strip()  # Réponse Q2\n        return reponse1, reponse2\n    else:\n        return None, None\n\n# Ajouter des colonnes pour les questions et réponses\ndf[\'question1\'] = "Q1: Pouvez-vous nous apporter un témoignage sur la façon dont votre structure répond, à travers ses modalités de fonctionnement, au contexte actuel et/ou celui de son émergence ?"\ndf[\'reponse1\'] = ""\ndf[\'qu

In [9]:
import pandas as pd
import re
from difflib import get_close_matches

# Charger la base de données à partir de l'Excel
df = pd.read_excel('spacestri.xlsx')

# Charger le fichier texte extrait du PDF
with open('extracted_text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Fonction pour extraire les noms du texte PDF sans les dates
def extraire_noms(texte):
    # Expression régulière pour extraire les noms suivis de dates entre parenthèses
    pattern = r'([A-Za-z0-9\s\+\-\(\)éÉû\-&’]*)\s*\((\d{4}\s*-\s*\d{0,4}|\d{4}\s*-\s* )\)'
    matches = re.findall(pattern, texte)
    # Retourner seulement les noms sans les dates
    noms = [match[0].strip() for match in matches if match[0].strip()]
    return noms

# Extraire les noms seuls du texte PDF
noms_pdf = extraire_noms(text)

# Fonction pour normaliser les noms
def normaliser_nom(nom):
    # Normaliser le nom pour qu'il commence par une majuscule et que le reste soit en minuscules
    return ' '.join(word.capitalize() for word in nom.split())

# Normaliser les noms extraits
noms_pdf_normalises = [normaliser_nom(re.sub(r'\s+', ' ', nom.lower()).strip()) for nom in noms_pdf]

# Afficher les noms extraits pour vérification
print("Noms extraits du texte PDF :")
for nom in noms_pdf_normalises:
    print(f"- {nom}")

# Normaliser les noms dans le DataFrame
df['nom_normalise'] = df['nom'].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

# Liste pour stocker les noms qui correspondent
noms_correspondants = []

# Parcourir chaque ligne du DataFrame pour associer un nom extrait du PDF
for index, row in df.iterrows():
    nom_excel = row['nom_normalise']
    
    # Correspondance floue avec les noms extraits du PDF
    matched_nom = get_close_matches(nom_excel, noms_pdf_normalises, n=1, cutoff=0.5)  # Seuil de correspondance à ajuster
    
    if matched_nom:  # Si une correspondance est trouvée
        if matched_nom[0] not in noms_correspondants:  # Vérifier si le nom n'a pas déjà été ajouté
            noms_correspondants.append(matched_nom[0])  # Ajouter la première correspondance trouvée
            print(f"Correspondance trouvée pour le nom : '{matched_nom[0]}'")
    #else:
        #print(f"Aucune correspondance trouvée pour le nom '{row['nom']}'")

# Afficher les correspondances trouvées
print("\nPremières correspondances trouvées :")
for nom in noms_correspondants:
    print(f"- {nom}")


Noms extraits du texte PDF :
- Immanence
- Particules
- Le Journal Des Expositions
- L’épicerie (1998 - 2004) Donnant-donnant (2005 - 2007) L’espace D’en Bas (2009 - 2019) Les Éditions De L’espace D’en Bas (2009 - ) Collection Adresse
- Pamela Artist-run Space
- Peepspace
- Kurt-forever (2011 - ) 6b
- Display
- Palais Des Paris
- Lieu-commun
- Glassbox
- Org (2013 - ) Échotone
- Poteaux D’angle
- Red Cactus Art Studio
- Glassbox-nord (1997 - ) Glassbox-sud
- Dos Mares
Correspondance trouvée pour le nom : 'Display'
Correspondance trouvée pour le nom : 'Immanence'
Correspondance trouvée pour le nom : 'Particules'
Correspondance trouvée pour le nom : 'Peepspace'
Correspondance trouvée pour le nom : 'Le Journal Des Expositions'
Correspondance trouvée pour le nom : 'Poteaux D’angle'
Correspondance trouvée pour le nom : 'Lieu-commun'
Correspondance trouvée pour le nom : 'Dos Mares'
Correspondance trouvée pour le nom : 'Glassbox'
Correspondance trouvée pour le nom : 'Red Cactus Art Studio'
Co

In [7]:
import pandas as pd

# Fonction pour extraire les données sans séparation supplémentaire
def extract_data_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]
    
    data = []
    
    # Parcourir les lignes par blocs de 5 : Nom, Q1, R1, Q2, R2
    for i in range(0, len(lines), 5):
        if i + 4 < len(lines):
            name = lines[i]       # Nom du répondant
            q1 = lines[i + 1]     # Question 1
            r1 = lines[i + 2]     # Réponse 1
            q2 = lines[i + 3]     # Question 2
            r2 = lines[i + 4]     # Réponse 2
            
            # Ajouter les données sous forme de dictionnaire
            data.append({
                'Nom': name,
                'Q1': q1,
                'R1': r1,
                'Q2': q2,
                'R2': r2
            })
    
    return pd.DataFrame(data)

# Chemin vers votre fichier texte
file_path = 'extracted_text.txt'

# Extraire les données et créer le DataFrame
df = extract_data_from_txt(file_path)

# Sauvegarder en fichier Excel
output_file = 'questionnaire_data.xlsx'
df.to_excel(output_file, index=False)

print(f'Données sauvegardées dans {output_file}')


Données sauvegardées dans questionnaire_data.xlsx
